In [73]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests

driver = webdriver.Firefox()
YEAR = str(2020)
try:
    driver.get('https://www.oyez.org/cases/' + YEAR)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href^="cases/'+ YEAR +'/"]'))
    )
except:
    driver.quit()

In [79]:
def repair(string):
    string = BeautifulSoup(response['facts_of_the_case']).text
    string = string.replace(u'\xa0', u' ')
    return string

DATE_TYPES = ['granted', 'argued', 'decided', 'citation']

elements = driver.find_elements(By.CSS_SELECTOR, 'ul.index.ng-scope li.ng-scope')
cases = []
for ele in elements:
    info = {}
    title = ele.find_element(By.TAG_NAME, 'h2')
    link = ele.find_element(By.CSS_SELECTOR, 'a[href^="cases/'+ YEAR +'/"]')
    id = link.get_attribute('href').split('/')[-1]
    description = ele.find_element(By.CLASS_NAME, 'description')
    dates = []
    for date in DATE_TYPES:
        val = ele.find_element(By.CLASS_NAME, date).find_elements(By.TAG_NAME, 'div')[0]
        if val == '':
            val = ele.find_element(By.CLASS_NAME, date).find_elements(By.TAG_NAME, 'div')[-1]
        dates.append(val.text)
    info.update({'title': title.text, 'id': id, 'description': description.text, 'dates': dates})
    response = requests.get('https://api.oyez.org/cases/'+YEAR+'/'+id).json()
    info.update({'petitioner': response['first_party'], 'respondent': response['second_party'], 'lower_court': response['lower_court']['name'], 'facts': repair(response['facts_of_the_case']), 'question': repair(response['question'])})
    cases.append(info)
print(cases)

decision on February 9, 2007.\nNearly seven years later, in 2013, Salinas filed a new application for a disability annuity. The Board granted him an annuity, but Salinas appealed the annuity&#39;s beginning date and amount. During that appeal, Salinas asked the Board to reopen all its decisions on his prior applications, including the decision denying his 2006 application. After a hearing, a Board hearing officer concluded that Salinas&#39;s 2006 application was beyond the four-year timeframe for reopening based on new and material evidence or administrative error under the Board&#39;s regulations. Salinas then asked the U.S. Court of Appeals to review the Board&#39;s decision not to reopen his 2006 application. Following its own binding precedent holding that it lacked jurisdiction to review a Board decision declining to reopen a prior benefits claim, the Fifth Circuit dismissed Salinas’s petition.\n&quot;}, {&#39;title&#39;: &#39;Torres v. Madrid&#39;, &#39;id&#39;: &#39;19-292&#39;,

In [68]:
driver.quit()

In [70]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("../firebase-key.json")
firebase_admin.initialize_app(cred, {
  'projectId': 'thesupremehacks',
})

db = firestore.client()

cases_collec = db.collection('cases')
small = [cases[0]]
for case in cases:
    case_ref = cases_collec.document(case['id']).set(case, merge=True)